In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

# Distribution of round-off errors

One way to get a sense of round-off errors is take a bunch of random numbers at high precision and store them at lower precision, e.g. take 64-bit floats and store as 32-bit floats.  We then make a histogram of the differences.  Obviously, random numbers generated as 64-bit floats will have round-off errors too, but they will be much smaller than the 32-bit errors so we can neglect them.

In [ ]:
values_64 = np.random.random(size=100000)
values_32 = np.float32(values_64)
diff = np.float64(values_32) - values_64

In [ ]:
hist = plt.hist(diff*1.e8, bins=100)
plt.xlabel(r'$(x_{32}-x_{64})/10^{-8}$',fontsize=15)

We also see that as the absolute errors are larger for large values, while the percentage errors are roughly the same.

In [ ]:
plt.plot(values_64, diff*1.e8, ',')
plt.xlabel(r'$x_{64}$',fontsize=15)
plt.ylabel(r'$(x_{32}-x_{64})/10^{-8}$',fontsize=15)

In [ ]:
plt.plot(values_64, diff/values_64*1e8, ',')
plt.xlabel(r'$x_{64}$',fontsize=15)
plt.ylabel(r'$(x_{32}-x_{64})/x_{64}/10^{-8}$',fontsize=15)

# Round-off vs. Approximation Errors

We test the simple algorithm for integrating $$\int_0^1 dx\,(x-x^2+x^3)$$ where the indefinite integral is given in the notes.  First we define subroutines for the function and its analytical formula for the definite integral.

In [ ]:
def fnc(x=None):
    return (x - x**2 + x**3)

In [ ]:
def fnc_int(limits=None):
    return ((limits[1]**2 - limits[0]**2) / 2. - 
            (limits[1]**3 - limits[0]**3) / 3. + 
            (limits[1]**4 - limits[0]**4) / 4.)

Now we define the integrating algorithm

In [ ]:
def integrate_fnc(limits=None, nstep=None, fnc=None, dtype=None):
    instep = 1. / np.float64(nstep).astype(dtype)
    step = (limits[1] - limits[0]) * instep # define the step size
    middles = limits[0] + (np.arange(nstep, dtype=dtype) + 0.5) * instep * (limits[1] - limits[0]) # set the interval x values
    values = fnc(middles)
    return (values.sum() * step) # add the terms

and a subroutine to do the testing.  This will produce the error as a function of the number of steps.

In [ ]:
def test_integrate(dtype=None):
    # produces a tuple with the number of steps along with the error for each number of steps
    limits = np.array([0., 1.], dtype=dtype)
    nstep_low = 10
    nstep_high = 100000000
    nnsteps = 50
    nsteps = 10.**(np.log10(nstep_low) +
                   (np.log10(nstep_high) - np.log10(nstep_low)) * np.arange(nnsteps) /
                   np.float64(nnsteps - 1)) # set the sampled Nsteps
    nsteps = np.int64(nsteps)
    error = np.zeros(nnsteps, dtype=np.float64)
    for indx in np.arange(nnsteps):
        integral = fnc_int(limits=np.float64(limits)) # compute the analytical result
        approx = integrate_fnc(limits=limits, nstep=nsteps[indx], fnc=fnc, dtype=dtype) # compute the approx result
        error[indx] = (approx - integral) / integral # compute the error
    return(nsteps, error)

Here are the results.

In [ ]:
(nsteps64, error64) = test_integrate(dtype=np.float64)
(nsteps32, error32) = test_integrate(dtype=np.float32)

In [ ]:
plt.plot(np.log10(nsteps64), np.log10(np.abs(error64)))
plt.plot(np.log10(nsteps32), np.log10(np.abs(error32)))
plt.ylabel('log$_{10}$ |$\epsilon$ / F|')
plt.xlabel('log$_{10}$ N$_{steps}$')

# Another Example: Powers of the Golden Ratio

The Golden Ratio $\phi=(\sqrt{5}-1)/2=0.618$ obeys the recursion relation $\phi^{n+1}=\phi^{n-1}-\phi^n$.  Here we demonstrate its instability.

In [ ]:
def golden_recurs(N=None,dtype=None):
    #uses the recursion relation
    phi=np.zeros(N+1,dtype=dtype)
    phi[0]=1.
    phi[1]=(np.sqrt(5,dtype=dtype)-1.)/2.
    for i in np.arange(2,N+1):
        phi[i]=phi[i-2]-phi[i-1]
    phi=phi[2:]
    return phi

In [ ]:
def golden_multiply(N=None,dtype=None):
    #computes it directly
    phi=np.zeros(N+1,dtype=dtype)
    phi[0]=1.
    phi[1]=(np.sqrt(5,dtype=dtype)-1.)/2.
    for i in np.arange(2,N+1):
        phi[i]=phi[i-1]*phi[1]
    phi=phi[2:]
    return phi

In [ ]:
def test_golden(dtype=None):
    #computes error
    Nmax=30
    phi=golden_recurs(N=Nmax,dtype=dtype)
    phi0=golden_multiply(N=Nmax,dtype=np.float64)
    N=np.arange(2,Nmax+1)
    error=(phi-phi0)/phi0
    return (N,error)

Here are the results.

In [ ]:
(N32,error32)=test_golden(dtype=np.float32)
(N64,error64)=test_golden(dtype=np.float64)

In [ ]:
print(error64)

In [ ]:
plt.plot(N64, np.log10(np.abs(error64)))
plt.plot(N32, np.log10(np.abs(error32)))
plt.ylabel('log$_{10}$ |$\epsilon$ / F|')
plt.xlabel('N')

Another way to test the stabilty of the forward recursion is to observe the difference between starting with $\phi_0=1$ and $\phi_1=0$ vs.~$\phi_0=0$ and $\phi_1=1$.

In [ ]:
def golden_recurs2(N=None,phi0=None,phi1=None,dtype=None):
    #uses the recursion relation
    phi=np.zeros(N+1,dtype=dtype)
    phi[0]=phi0
    phi[1]=phi1
    for i in np.arange(2,N+1):
        phi[i]=phi[i-2]-phi[i-1]
    phi=phi[2:]
    return phi

In [ ]:
def test_golden2(dtype=None):
    #computes error
    Nmax=30
    phi01=golden_recurs2(N=Nmax,phi0=0.,phi1=1.,dtype=dtype)
    phi10=golden_recurs2(N=Nmax,phi0=1.,phi1=0.,dtype=dtype)
    N=np.arange(2,Nmax+1)
    error=abs(phi01-phi10)
    return (N,error)

In [ ]:
N,phidiff=test_golden2(dtype=np.float64)
plt.plot(N, np.log10(phidiff))
#plt.plot(N32, np.log10(np.abs(error32)))
plt.ylabel('log$_{10}$ |$\Delta\phi$|')
plt.xlabel('N')

The rapid increase of the difference between the two recursions shows it is unstable.

Here we demonstrate Miller's algorithm as a solution to this problem.

In [ ]:
def golden_millers(N=None,dtype=None):
    phi=np.zeros(N+1,dtype=dtype)
    phi[N]=1.
    phi[N-1]=1.
    for i in np.arange(N-2,0,-1):
        phi[i]=phi[i+1]+phi[i+2]
    phi*=(np.sqrt(5,dtype=dtype)-1.)/2./phi[1]
    phi=phi[2:]
    return phi

In [ ]:
def test_golden2(dtype=None):
    Nmax=30
    phi=golden_millers(N=Nmax,dtype=dtype)
    phi0=golden_multiply(N=Nmax,dtype=np.float64)
    N=np.arange(2,Nmax+1)
    error=(phi-phi0)/phi0
    return (N,error)

Here are the results.

In [ ]:
(N32,error32)=test_golden2(dtype=np.float32)
(N64,error64)=test_golden2(dtype=np.float64)

In [ ]:
plt.plot(N64, np.log10(np.abs(error64)))
plt.plot(N32, np.log10(np.abs(error32)))
plt.ylabel('log$_{10}$ |$\epsilon$ / F|')
plt.xlabel('N')

There is still a limit to Miller's theorem, in that you can't go lower than machine precision.

# Power Series Approximation

Here we evaluate the power series for $\cos x$, given by $$\cos x=\sum_{n=0}^\infty \frac{(-1)^nx^{2n}}{(2n)!}=1-\frac{x^2}{2!}+\frac{x^4}{4!}-\frac{x^6}{6!}+...$$

We compare summing directly with summing using the formula $$n^{\rm th}\,{\rm term}=-\frac{x^2}{(2n)(2n-1)}(n-1)^{\rm th}\,{\rm term}$$

We also choose the tolerance $10^{-8}$.

In [ ]:
def cos_direct(x=None,tol=None):
    cos=np.float32(1.)
    n=0
    diff=100.
    while diff > tol:
        n+=1
        newterm=(-1)**n*x**(2*n)/np.math.factorial(2*n)
        diff=np.absolute(newterm/cos)
        cos+=newterm
    return (cos,n)

In [ ]:
def cos_nrat(x=None,tol=None):
    cos=np.float32(1.)
    newterm=cos
    n=0
    diff=100.
    while diff > tol:
        n+=1
        newterm*=-x**2/((2*n)*(2*n-1))
        diff=np.absolute(newterm/cos)
        cos+=newterm
    return (cos,n)

In [ ]:
def test_cos(func=None,x=None,tol=None):
    cos,n=func(x=x,tol=tol)
    cos0=np.cos(x,dtype=np.float64)
    error=(cos-cos0)/cos0
    return (error,n)

$x=10$ gives the same error for both methods

In [ ]:
x=np.float32(10.)
print(test_cos(cos_direct,x,tol=1.e-8))
print(test_cos(cos_nrat,x,tol=1.e-8))

while $x=20$ clearly does better using the method with the ratios.

In [ ]:
x=np.float32(20.)
print(test_cos(cos_direct,x,tol=1.e-8))
print(test_cos(cos_nrat,x,tol=1.e-8))

In [ ]:
cos1=np.float32(1.)
cos2=np.float32(1.)
newterm1=1.
newterm2=1.
diff1=np.zeros(40)
diff2=np.zeros(40)
x=np.float32(20.)
for n in np.arange(1,40):    
        newterm1=(-1)**n*x**(2*n)/np.math.factorial(2*n)
        newterm2*=-x**2/((2*n)*(2*n-1))
        cos1+=newterm1
        cos2+=newterm2
        costrue=np.cos(x)
        diff1[n]=np.absolute(cos1-costrue)/costrue
        diff2[n]=np.absolute(cos2-costrue)/costrue

In [ ]:
N=np.arange(40)
plt.plot(N, np.log10(np.abs(diff1)))
plt.plot(N, np.log10(np.abs(diff2)))
plt.ylabel('log$_{10}$ |$\epsilon$ / F|')
plt.xlabel('N')